# Convert pret model to model_4ch

In [ ]:
%ls /home/models/w2v/w2v_pret_chunks64000_ts_4s_newds_4M_v1/

In [ ]:
# BASE
cp_path = '/home/models/w2v/w2v_pret_chunks64000_ts_4s_newds_4M_v1/checkpoint_165_400000_fixed.pt'
cp_save = '/home/models/w2v/w2v_pret_chunks64000_ts_4s_newds_4M_v1/ckpt_165_400000_l3das21.pt'

# LARGE
# cp_path = '/home/models/w2v/w2v_pret_chunks64000_ts_4s_newds2_600K_large_norm_v1/checkpoint_148_600000_fixed.pt'
# cp_save = '/home/models/w2v/w2v_pret_chunks64000_ts_4s_newds2_600K_large_norm_v1/ckpt_148_600000_l3das22.pt'

In [ ]:
import torch
import fairseq
from argparse import ArgumentParser
from collections import namedtuple

user_dir = "/home/seld_wav2vec2/src/seld_wav2vec2"

Arg = namedtuple("Arg", ["user_dir"])
arg = Arg(user_dir.__str__())
fairseq.utils.import_user_module(arg)

model_pret, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path], strict=False)
model = model_pret[0]
model.eval()

# model_pret = model_pret.cuda()

In [ ]:
cfg

In [ ]:
cfg.model

In [ ]:
from omegaconf.dictconfig import DictConfig

w2v_state = model.state_dict()
content = dict(cfg.model)

print(content)

content["_name"] = 'wav2vec2_ch'

res = DictConfig(content)

print(res)

In [ ]:
model_4ch = task.build_model(res, from_checkpoint=True)

state = fairseq.checkpoint_utils.load_checkpoint_to_cpu(cp_path)

In [ ]:
print(state.keys())

print(state['args'])

print(state['cfg'])

In [ ]:
state_dict = model_4ch.state_dict()
state_model = state["model"].copy()
for key in state["model"]:
    if key in state_dict.keys():
        if state["model"][key].shape != state_dict[key].shape:
            print("key {} is not matching".format(key))
            state_model.pop(key)

model_4ch.load_state_dict(state_model, strict=False)

print(state['args'])

# state['args'] = {"arch": 'wav2vec2_ch'}
state['model'] = model_4ch.state_dict()
state['cfg']['model'] = res


!rm {cp_save}

torch.save(state, cp_save)

In [ ]:
import torch
import fairseq
from argparse import ArgumentParser
from collections import namedtuple

user_dir = "/home/seld_wav2vec2/src/seld_wav2vec2"

Arg = namedtuple("Arg", ["user_dir"])
arg = Arg(user_dir.__str__())
fairseq.utils.import_user_module(arg)

model_u, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_save], strict=True)
model_u = model_u[0]
model_u.eval()

# saved_4ch_w2v_state = model_u.state_dict()

# for key in w2v_state.keys():
#     if not torch.equal(w2v_state[key], saved_4ch_w2v_state[key]):
#         print("key", key)

In [ ]:
state_saved = fairseq.checkpoint_utils.load_checkpoint_to_cpu(cp_save)

print(state_saved.keys())